In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
import requests
import yaml
import json
import os
import pandas as pd
import gzip
import shutil

In [36]:
from gdc.download import api_download_iterative

Config

In [37]:
%ls conf/

Icon?           __init__.py     user_conf.yaml


In [38]:
with open('conf/user_conf.yaml', 'r') as f:
    conf = yaml.load(f)

Paths

In [39]:
slides_metadata_file = os.path.join(conf['data_path'], 'slides_metadata.csv')
rnaseq_metadata_file = os.path.join(conf['data_path'], 'rnaseq_metadata.csv')

In [40]:
slides_path = os.path.join(conf['data_path'], 'slides')
if not os.path.exists(slides_path):
    os.mkdir(slides_path)

In [41]:
rnaseq_path = os.path.join(conf['data_path'], 'rnaseq')

if not os.path.exists(rnaseq_path):
    os.mkdir(rnaseq_path)

In [42]:
# Temporal

slides_path = os.path.join(conf['data_path'], 'diag_slides')
if not os.path.exists(slides_path):
    os.mkdir(slides_path)

# Download Files

Read metadata

In [43]:
slides_df = pd.read_csv(slides_metadata_file, sep='|')
slides_df = slides_df[['file_name', 'file_id', 'file_size', 'sample_id', 'experimental_strategy']]
slides_df.head(3)

,file_name,file_id,file_size,sample_id,experimental_strategy
0,TCGA-2L-AAQM-11A-01-TSA.svs,ddce0ab5-47b7-43c2-8d67-3d50c48d8ae7,60.99,TCGA-2L-AAQM-11A,Tissue Slide
1,TCGA-HZ-A49I-01A-01-TS1.svs,7ec5c9d0-8c22-4e30-bae8-605c3dcfe744,293.72,TCGA-HZ-A49I-01A,Tissue Slide
2,TCGA-3A-A9IZ-01A-01-TS1.svs,265adb6b-913e-490c-8cea-32d67c1440a2,166.95,TCGA-3A-A9IZ-01A,Tissue Slide


In [44]:
rnaseq_df = pd.read_csv(rnaseq_metadata_file, sep='|')
rnaseq_df = rnaseq_df[['file_name', 'file_id', 'file_size', 'sample_id', 'workflow_type']]
rnaseq_df.head(3)

,file_name,file_id,file_size,sample_id,workflow_type
0,TCGA-FB-AAPS-01A_HTSeq-FPKM-UQ.txt.gz,1e0be26b-da47-4e56-926c-9afc259a2bd0,0.53,TCGA-FB-AAPS-01A,HTSeq - FPKM-UQ
1,TCGA-FB-AAQ3-01A_HTSeq-FPKM-UQ.txt.gz,2803abb9-bf21-4b75-a3c9-687c87b75701,0.50,TCGA-FB-AAQ3-01A,HTSeq - FPKM-UQ
2,TCGA-HZ-8005-01A_HTSeq-Counts.txt.gz,4d98a852-c30a-437b-a214-ee81015d674b,0.25,TCGA-HZ-8005-01A,HTSeq - Counts


## Slides

*Keep only the ones with RNASeq related data?*
*Keep only tissue slides?*

Number of files and count

In [45]:
summary = slides_df.groupby('experimental_strategy').agg({'file_name': 'size', 'file_size': 'sum'})
summary = summary.rename(columns={'file_name': 'count', 'file_size': 'total_size (gb)'})
summary['total_size (gb)'] = round(summary['total_size (gb)'] / 1000, 2)

summary

,count,total_size (gb)
experimental_strategy,,
Diagnostic Slide,209,244.53
Tissue Slide,257,57.48


In [46]:
rna_seq_samples = rnaseq_df['sample_id'].unique()

In [47]:
# slides_df = slides_df[slides_df['sample_id'].isin(rna_seq_samples)]
slides_df = slides_df[slides_df['experimental_strategy'] == 'Diagnostic Slide']

In [48]:
slides_df = slides_df.sample(n=20)

In [49]:
summary = slides_df.groupby('experimental_strategy').agg({'file_name': 'size', 'file_size': 'sum'})
summary = summary.rename(columns={'file_name': 'count', 'file_size': 'total_size (gb)'})
summary['total_size (gb)'] = round(summary['total_size (gb)'] / 1000, 2)

summary

,count,total_size (gb)
experimental_strategy,,
Diagnostic Slide,20,26.16


In [50]:
# Download only failed
files_downloaded = os.listdir(slides_path)
slides_df = slides_df[-slides_df['file_name'].isin(files_downloaded)]

In [51]:
summary = slides_df.groupby('experimental_strategy').agg({'file_name': 'size', 'file_size': 'sum'})
summary = summary.rename(columns={'file_name': 'count', 'file_size': 'total_size (gb)'})
summary['total_size (gb)'] = round(summary['total_size (gb)'] / 1000, 2)

summary

,count,total_size (gb)
experimental_strategy,,
Diagnostic Slide,20,26.16


In [56]:
slides_df = slides_df.iloc[2:]

In [57]:
api_download_iterative(slides_df, slides_path, multiprocess=4)

Process ForkPoolWorker-7:
Process ForkPoolWorker-8:
Process ForkPoolWorker-9:
Process ForkPoolWorker-10:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/portizdegalisteo/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/portizdegalisteo/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/portizdegalisteo/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/portizdegalisteo/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/portizdegalisteo/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/portizdegalisteo/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._t

KeyboardInterrupt: 

## RNA-Seq

In [119]:
summary = rnaseq_df.groupby('workflow_type').size()
summary.reset_index()

,workflow_type,0
0,HTSeq - Counts,182
1,HTSeq - FPKM,182
2,HTSeq - FPKM-UQ,182


In [32]:
summary = rnaseq_df.groupby('workflow_type').agg({'file_name': 'size', 'file_size': 'sum'})
summary = summary.rename(columns={'file_name': 'count', 'file_size': 'total_size (gb)'})
summary['total_size (gb)'] = round(summary['total_size (gb)'] / 1000, 2)

summary

,count,total_size (gb)
workflow_type,,
HTSeq - Counts,182,0.05
HTSeq - FPKM,182,0.09
HTSeq - FPKM-UQ,182,0.09


In [34]:
slides_samples = slides_df['sample_id'].unique()

In [36]:
rnaseq_df = rnaseq_df[rnaseq_df['sample_id'].isin(slides_samples)]

In [37]:
summary = rnaseq_df.groupby('workflow_type').agg({'file_name': 'size', 'file_size': 'sum'})
summary = summary.rename(columns={'file_name': 'count', 'file_size': 'total_size (gb)'})
summary['total_size (gb)'] = round(summary['total_size (gb)'] / 1000, 2)

summary

,count,total_size (gb)
workflow_type,,
HTSeq - Counts,182,0.05
HTSeq - FPKM,182,0.09
HTSeq - FPKM-UQ,182,0.09


In [39]:
files = api_download_iterative(rnaseq_df, rnaseq_path, multiprocess=8)

['TCGA-HZ-8005-01A_HTSeq-Counts.txt.gz',
 'TCGA-HV-A7OL-01A_HTSeq-FPKM.txt.gz',
 'TCGA-FB-AAPQ-01A_HTSeq-FPKM.txt.gz',
 'TCGA-FB-AAPS-01A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-3A-A9IB-01A_HTSeq-FPKM.txt.gz',
 'TCGA-FB-AAQ3-01A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-IB-AAUU-01A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-H6-A45N-11A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-PZ-A5RE-01A_HTSeq-Counts.txt.gz',
 'TCGA-IB-7891-01A_HTSeq-Counts.txt.gz',
 'TCGA-2L-AAQE-01A_HTSeq-FPKM.txt.gz',
 'TCGA-F2-A7TX-01A_HTSeq-Counts.txt.gz',
 'TCGA-OE-A75W-01A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-IB-AAUM-01A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-IB-7649-01A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-HZ-7289-01A_HTSeq-Counts.txt.gz',
 'TCGA-IB-AAUR-01A_HTSeq-Counts.txt.gz',
 'TCGA-3A-A9IO-01A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-IB-A7LX-01A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-IB-A5SP-01A_HTSeq-FPKM.txt.gz',
 'TCGA-2L-AAQM-01A_HTSeq-Counts.txt.gz',
 'TCGA-HZ-8636-01A_HTSeq-FPKM.txt.gz',
 'TCGA-3A-A9I7-01A_HTSeq-Counts.txt.gz',
 'TCGA-3A-A9IH-01A_HTSeq-FPKM-UQ.txt.gz',
 'TCGA-3A-A9IU-01A

In [42]:
def gunzip(source_filepath, dest_filepath):
    with gzip.open(source_filepath, 'rb') as s_file:
        with open(dest_filepath, 'wb') as d_file:
            shutil.copyfileobj(s_file, d_file)

In [51]:
for file_name in files:
    
    source_filepath = os.path.join(rnaseq_path, file_name)
    dest_filepath = re.sub('\.gz$', '', source_filepath)
    
    gunzip(source_filepath, dest_filepath)
    os.remove(source_filepath)